# Pip Installs for Quantization
In order to squeeze better performance out of our models, we want to make use of some quanitzation techniques. I'm just now learning about these so I'll be trying a few out. Source: https://generativeai.pub/practical-guide-of-llm-quantization-gptq-awq-bitsandbytes-and-unsloth-bdeaa2c0bbf6

In [14]:
!pip install auto-gptq trl bitsandbytes>=0.39.0 datasets -q
!pip install accelerate transformers optimum onnx onnxruntime -q
!pip uninstall transformers -y
!pip install transformers==4.46.1 -q

!pip uninstall huggingface_hub -y
!pip install huggingface_hub==0.25 -q

!pip install scikit-learn matplotlib -q

!pip install flash-attn --no-build-isolation -q
!pip install -U transformers unsloth bitsandbytes flash-attn -q
!pip install git+https://github.com/unslothai/unsloth.git -q


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Found existing installation: transformers 4.50.3
Uninstalling transformers-4.50.3:
  Successfully uninstalled transformers-4.50.3

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Found existing installation: huggingface-hub 0.30.1
Uninstalling huggingface-hub-0.30.1:
  Successfully uninstalled huggingface-hub-0.30.1

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[not

# Google and HuggingFace Drive Boilerplate
This has boilerplate code which connects our collab to google drive. This allows us to persist some of the data that we create & to store large datasets

In [2]:
import os

try:
  import google.colab
  from google.colab import drive
  # drive.mount('/content/drive')
except:
  print("Not in colab, just loading hugging face")


from huggingface_hub import login, HfApi


Not in colab, just loading hugging face


## Upload To Hugging Face

In [3]:
import pandas as pd

data_path = '/content/drive/My Drive/persuade_corpus_2.0_train.csv'
repo_id = "ErikNordby/EdTech"  # Replace with your username and desired dataset name

UPLOAD = False
if UPLOAD:
    api = HfApi(token=os.environ.get("HUGGINGFACE_TOKEN"))
    api.upload_file(
        path_or_fileobj=data_path,
        path_in_repo="persuade_corpus_2.0_train.csv",  # Name of the file in the repository
        repo_id=repo_id,
        repo_type="dataset"
    )




## Download from Hugging Face

In [4]:
from huggingface_hub import hf_hub_download
import pandas as pd

# Specify the repository information
repo_id = "ErikNordby/EdTech"  # Replace with the actual repository
filename = "persuade_corpus_2.0_train.csv"  # Replace with the actual filename in the repository

# Download the file
login(token=os.environ.get("HUGGINGFACE_TOKEN"))
file_path = hf_hub_download(repo_id=repo_id, filename=filename, repo_type="dataset")

# Load the CSV into a pandas DataFrame
essays = pd.read_csv(file_path)


persuade_corpus_2.0_train.csv:   0%|          | 0.00/617M [00:00<?, ?B/s]

/tmp/ipykernel_3373/1655391147.py:13: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  essays = pd.read_csv(file_path)


# Create Dataset with Persuade Dataset

The persuade dataset contains essays written by students from grades 8-12. This section loads up those essays & creates 2 different versions of prompts. First we create prompts with just the plain data & a request for feedback. Secondly, we create a dataset which asks for feedback which follows the criteria put forth for good feedback by Macfarlane-Dick (Cite Here)

## Clean Data
Here we're just cleaning up the data since it's a pretty large dataset. For right now, we're just going to try getting away with 10 essays which are relatively short

In [5]:
short_essays = essays[essays['essay_word_count'] < 500]
short_essays = short_essays.sample(n=5)
short_essays = short_essays.reset_index()
short_essays_text = short_essays['full_text']

## Create Prompts
Here we're creating the prompts that we're going to be using. Currently, I'm just using some pretty simple example which break out the desired behavior into different types. The thought is that maybe we can isolate these behaviors into different vectors which can be combined or compared to a more general "good advice" prompt set. Note: This is for my course that I'm taking, leaving this in but not using the essay prompts

In [6]:
def apply_feedback_prompts(dataset, feedback_prompts):
  essay_prompts = []
  for feedback_prompt in feedback_prompts:
    prompts = short_essays_text.apply(lambda x: "Essay: " + x.strip() + ' Request: ' + feedback_prompt)
    essay_prompts.append(prompts)
  return pd.concat(essay_prompts).values

negative_examples = [
    "Critique this harshly",
    "Give mean feedback",
    "Critique this",
    "Tear this apart with your feedback",
    "Be brutally honest about the flaws",
    "Don't hold back any criticism",
    "Give your most ruthless feedback",
    "Point out everything wrong with this",
    "What's the harshest criticism you can give?",
    "Provide only negative feedback",
    "Destroy this with your critique",
    "Tell me why this is terrible",
    "What makes this completely unacceptable?",
    "Give feedback as if you hate this",
    "List all the failures in this work",
    "Be merciless in your assessment",
    "What would a harsh critic say?",
    "Criticize this as severely as possible",
    "Tell me every single flaw you can find",
    "What's the worst thing about this?"
]

encouraging_examples = [
    "Give good feedback for this which is supportive",
    "Give good feedback which is encouraging",
    "Give feedback which highlights areas of success",
    "Provide constructive feedback with a positive tone",
    "Share supportive comments that build confidence",
    "Offer feedback that recognizes strengths while suggesting improvements",
    "Give encouraging feedback that inspires continued effort",
    "Highlight what works well in this",
    "Share positive reinforcement with gentle suggestions",
    "Provide uplifting feedback that acknowledges the effort",
    "What's particularly impressive about this?",
    "Tell me what you love about this work",
    "Give feedback that celebrates the achievements here",
    "Share what makes this stand out in a positive way",
    "Offer encouragement that builds on existing strengths",
    "What would make someone feel proud about this work?",
    "Give feedback as a supportive mentor would",
    "Share what's most promising about this approach",
    "How would you compliment this work to boost confidence?",
    "Provide feedback that appreciates the creativity shown"
]


# Bad examples
negative_essay_prompts = apply_feedback_prompts(short_essays_text, negative_examples).tolist()

# Good examples
encourageing_essay_prompts = apply_feedback_prompts(short_essays_text, encouraging_examples).tolist()


# Create hooks for the activation
We want to be able to both have visibility into the activations and to also be able to modify the activations. So, we're creating hooks that can be attached to layers of the model which allow us to do that.

In [17]:
import torch
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm

torch.cuda.empty_cache()

class ActivationHook:
    """Hook to extract activations from a specific layer of a transformer model."""
    def __init__(self, module, layer_name, transformation_function=None):
        self.activations = None
        self.layer_name = layer_name
        self.transformation_function = transformation_function if transformation_function is not None else lambda x: x

        # Register forward hoo_k
        if "mlp" in self.layer_name:
            module.register_forward_hook(self.hook_mlp)
        elif "self_attn" in self.layer_name:
            module.register_forward_hook(self.hook_attention)
        elif "residual" in self.layer_name:
            module.register_forward_hook(self.hook_residual)
        else:
            raise ValueError(f"Unsupported layer: {layer_name}")

    def set_transformation_function(self, transformation_function):
        self.transformation_function = transformation_function

    def hook_mlp(self, module, input, output):
        self.activations = output.detach()
        return self.transformation_function(output)

    def hook_attention(self, module, input, output):
        self.activations = output[0].detach()
        output_updated = self.transformation_function(output[0])
        output = (output_updated,) + output[1:]
        return output

    def hook_residual(self, module, input, output):
        self.activations = output[0].detach()
        output_updated = self.transformation_function(output[0])
        output = (output_updated,) + output[1:]
        return output

    def print_activations(self):
        print(self.activations)

    def clear_activations(self):
        self.activations = None

    def clear_transformation_function(self):
        self.transformation_function = lambda x: x

    def clear(self):
        self.clear_activations()
        self.clear_transformation_function()

    def __del__(self):
        self.clear()




# Setting Up Steering Class
Now that we have our arrays of string and our code to hook the activation, we now need to load in the language model and write some code to actually handle the steering

# Setting Up Steering Code

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F # Import functional for softmax

from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig
from abc import ABC, abstractmethod
import gc

gc.collect()
torch.cuda.empty_cache()

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from accelerate import infer_auto_device_map, init_empty_weights, load_checkpoint_and_dispatch
from optimum.onnxruntime import ORTModelForCausalLM
from optimum.bettertransformer import BetterTransformer
from huggingface_hub import snapshot_download

class LLMSteerer:
    def __init__(self, model_name):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_quant_type="nf4",  # Higher quality quantization
            bnb_4bit_use_double_quant=True  # Further reduces memory usage
        )

        # Replace your model loading in LLMSteerer.__init__ with:
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=quantization_config,
            device_map="auto",
            attn_implementation="eager",  # Avoid Flash Attention error
            low_cpu_mem_usage=True
        )
        self.model.to(self.device)
        self.model.eval()

        # OR Option B: Use BetterTransformer for memory-efficient attention
        # self.model = BetterTransformer.transform(base_model)

    def generate(self, prompt, max_length=100):
        # Process input with optimized inference
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)

        # Use generate method of optimized model
        with torch.no_grad(), torch.cuda.amp.autocast():  # Use mixed precision
            outputs = self.model.generate(
                input_ids=inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_new_tokens=max_length,
                pad_token_id=self.tokenizer.eos_token_id
            )

        # Decode and return generated text
        generated_text = self.tokenizer.decode(
            outputs[0][inputs.input_ids.shape[1]:],
            skip_special_tokens=True
        )

        # Clean up to free memory
        torch.cuda.empty_cache()

        return generated_text


class PromptSteerer(LLMSteerer):
    def __init__(self, model, prompt_template, hidden_size=128):
        super().__init__(model, hidden_size)
        self.prompt_template = prompt_template

    def generate(self, prompt):
        full_prompt = self.prompt_template.format(prompt)
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)
        return self.model.generate(
            input_ids=inputs.input_ids.to(model.device),
            attention_mask=inputs.attention_mask.to(model.device) if 'attention_mask' in inputs else None,
            max_new_tokens=5,  # Generate a few tokens to catch the answer
            pad_token_id=self.tokenizer.eos_token_id
        )


class ActivationController(LLMSteerer):
    def __init__(self, model, selected_blocks=None, hidden_size=128):
        super().__init__(model)

        self.CACHE_DIR = "/content/my_caches"
        self.hooks = {}
        num_layers = self.model.config.num_hidden_layers

        # Dynamically create layer names for every 5th layer
        self.selected_layers = []
        for i in range(0, num_layers, 5):  # Step by 5
            # Add the last layer if we're not exactly divisible by 5
            if i >= num_layers:
                break
            self.selected_layers.append(f'model.layers.{i}.self_attn')
            self.selected_layers.append(f'model.layers.{i}.mlp')

        # Add the last layer if it wasn't included
        if (num_layers - 1) % 5 != 0:
            self.selected_layers.append(f'model.layers.{num_layers-1}.self_attn')
            self.selected_layers.append(f'model.layers.{num_layers-1}.mlp')

        if selected_blocks is not None:
            self.selected_layers = selected_blocks

        def get_layer(layer_name, layer_module):
            layer_idx = int(layer_name.split('.')[-2])
            if "GPTNeoXForCausalLM" in self.model.config.architectures:
                layer = self.model.gpt_neox.layers[layer_idx]
            else:
                layer = self.model.model.layers[layer_idx]

            if 'mlp' in layer_name:
                return layer.mlp
            elif 'self_attn' in layer_name:
                if "GPTNeoXForCausalLM" in self.model.config.architectures:
                    return layer.attention
                else:
                    return layer.self_attn
            elif 'residual' in layer_name:
                return layer

        # Add hooks
        for layer_name in self.selected_layers:
            if 'mlp' in layer_name:
                module = get_layer(layer_name, "mlp")
            elif 'self_attn' in layer_name:
                module = get_layer(layer_name, "self_attn")
            elif 'residual' in layer_name:
                module = get_layer(layer_name, "residual")
            else:
                raise ValueError(f"Unsupported layer: {layer_name}")

            self.hooks[layer_name] = ActivationHook(module, layer_name)

    def extract_activation(self, layer_name, hook, aggregation_calc):
        """Extracts and aggregates activation, minimizing CPU RAM usage."""
        if hook.activations is None:
            return None # Or handle appropriately

        activations_gpu = hook.activations # Keep on original device (likely GPU)

        # Perform aggregation on the GPU tensor
        if aggregation_calc == "mean":
            # Mean across sequence length (dim=1 assuming batch, seq, hidden)
            # Keepdims=True might be useful if subsequent code expects shape (batch, 1, hidden)
            aggregated_gpu = torch.mean(activations_gpu, dim=1)
        elif aggregation_calc == "max":
            # Max across sequence length
            aggregated_gpu = torch.max(activations_gpu, dim=1).values # .values is important for max
        elif aggregation_calc == "last":
            # Take the last token's activation along the sequence dim (dim=1)
            aggregated_gpu = activations_gpu[:, -1, :]
        else:
            # Default to last token or raise error
            print(f"Warning: Unsupported aggregation '{aggregation_calc}'. Defaulting to 'last'.")
            aggregated_gpu = activations_gpu[:, :, :]

        # Now move only the smaller, aggregated tensor to CPU and convert to numpy
        # Use detach() just in case gradients were somehow attached
        aggregated_cpu_numpy = aggregated_gpu.detach().cpu().numpy()

        return aggregated_cpu_numpy

    def extract_activations(self, texts, device='cuda', aggregation_calc="all"):
        """Extract activations from specified layers for a list of input texts."""
        # Extract activations for each text
        all_activations = {layer_name: [] for layer_name in self.selected_layers}

        for text_index in range(len(texts)):
            text = texts[text_index]

            # Tokenize and process through model
            inputs = self.tokenizer(text, return_tensors='pt')
            inputs = inputs.to(self.device)
            with torch.no_grad():
                self.model(**inputs)

            # Collect activations from hooks
            for layer_name, hook in self.hooks.items():
                if hook.activations is not None:
                    last_token_activation = self.extract_activation(layer_name, hook, aggregation_calc)
                    all_activations[layer_name].append(last_token_activation)
                    hook.clear()


        return all_activations


    def set_transformation_function(self, layer_name, transformation_function):
        self.hooks[layer_name].set_transformation_function(transformation_function)

    def get_transformation_function(self, layer_name):
        return self.hooks[layer_name].transformation_function

    def clear_hooks(self):
        for hook in self.hooks.values():
            hook.clear()

    def generate(self, prompt, max_length = 1):
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)
        generated = self.model.generate(
            input_ids=inputs.input_ids.to(self.model.device),
            attention_mask=inputs.attention_mask.to(self.model.device) if 'attention_mask' in inputs else None,
            max_new_tokens=max_length,  # Generate a few tokens to catch the answer
            pad_token_id=self.tokenizer.eos_token_id
        )


        input_length = inputs.input_ids.shape[1]
        generated_tokens = generated[0][input_length:]
        generated_text = self.tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
        return generated_text

    def __del__(self):
        self.clear_hooks()



# Averaging Method


In [19]:
class ActAddSteerer(ActivationController):
    def __init__(self, model, selected_blocks=None, hidden_size=128):
        super().__init__(model)

    # Inside the ActAddSteerer class
    def extract_average_activations(self, texts, device='cuda', aggregation_calc="last"):
        """Calculates average activations directly without storing all intermediates."""
        if not self.selected_layers:
            return {}

        # Use the modified extract_activation from Solution 1 above
        # Initialize sums and counts
        activation_sums = {layer_name: None for layer_name in self.selected_layers}
        activation_counts = {layer_name: 0 for layer_name in self.selected_layers}
        example_shape = {layer_name: None for layer_name in self.selected_layers} # To store shape info

        # Use tqdm for progress tracking if desired
        from tqdm.notebook import tqdm

        for text in tqdm(texts, desc="Extracting Avg Activations"):
            inputs = self.tokenizer(text, return_tensors='pt', truncation=True, max_length=self.model.config.max_position_embeddings // 2).to(device) # Add truncation
            try:
                with torch.no_grad():
                    _ = self.model(**inputs) # Run forward pass, hooks capture activations
            except Exception as e:
                print(f"Warning: Error processing text: {text[:50]}... Error: {e}")
                continue # Skip this text if model forward pass fails

            for layer_name, hook in self.hooks.items():
                # Use the MEMORY-EFFICIENT extract_activation from Solution 1
                aggregated_numpy = self.extract_activation(layer_name, hook, aggregation_calc)

                if aggregated_numpy is not None:
                    current_act_flat = aggregated_numpy.flatten() # Ensure 1D for summing

                    if activation_sums[layer_name] is None:
                        activation_sums[layer_name] = current_act_flat
                        example_shape[layer_name] = aggregated_numpy.shape # Store original shape
                    else:
                        # Ensure consistent shapes before adding
                        if activation_sums[layer_name].shape == current_act_flat.shape:
                            activation_sums[layer_name] += current_act_flat
                        else:
                            print(f"Warning: Shape mismatch for layer {layer_name}. Skipping update.")
                            continue # Skip if shapes don't match (shouldn't happen with consistent aggregation)

                    activation_counts[layer_name] += 1
                hook.clear() # Clear hook activation storage

        # Calculate averages
        average_activations = {}
        for layer_name in self.selected_layers:
            if activation_counts[layer_name] > 0:
                avg = activation_sums[layer_name] / activation_counts[layer_name]
                # Reshape back to original aggregated shape if needed (e.g., (1, hidden_dim))
                original_shape = example_shape[layer_name]
                if original_shape:
                    try:
                        average_activations[layer_name] = avg.reshape(original_shape)
                    except ValueError:
                        print(f"Warning: Could not reshape avg for layer {layer_name} back to {original_shape}. Keeping flat.")
                        average_activations[layer_name] = avg # Keep flat if reshape fails
                else:
                    average_activations[layer_name] = avg # Keep flat if shape wasn't stored
            else:
                print(f"Warning: No activations collected for layer {layer_name}. Returning None.")
                average_activations[layer_name] = None # Or np.zeros(...) if preferred

        # Clean up large intermediate sums immediately
        del activation_sums
        del activation_counts
        del example_shape
        torch.cuda.empty_cache()
        import gc
        gc.collect()

        return average_activations

    def extract_average_diff(self, positive_texts, negative_texts, device='cuda'):
        positive_activations = self.extract_average_activations(positive_texts, device)
        negative_activations = self.extract_average_activations(negative_texts, device)

        average_diff_activations = {}
        for layer_name, activations in positive_activations.items():
            average_diff_activations[layer_name] = positive_activations[layer_name] - negative_activations[layer_name]

        return average_diff_activations


# Create Plots

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import torch
from sklearn.manifold import TSNE
# from umap import UMAP # Uncomment if you have umap-learn installed and prefer UMAP
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from functools import partial # Useful for setting transformation functions

# Assuming your existing classes (ActivationHook, LLMSteerer, ActivationController, etc.)
# are defined above or imported.

# --- Visualization Functions ---

def plot_activation_projection(all_activations, layer_name, labels, method='tsne', perplexity=30, n_neighbors=15, min_dist=0.1, title_suffix=""):
    """
    Plots a 2D projection (t-SNE or UMAP) of activations for a specific layer,
    colored by labels (e.g., positive/negative).

    Args:
        all_activations (dict): Dictionary from layer_name to list of activations (numpy arrays).
                                Assumes each activation in the list is for one text input,
                                likely shape (hidden_dim,) or (1, hidden_dim).
        layer_name (str): The layer whose activations to plot.
        labels (list): A list of labels (e.g., 'positive', 'negative') corresponding
                       to the order of activations in all_activations[layer_name].
        method (str): 'tsne' or 'umap'.
        perplexity (float): Perplexity for t-SNE.
        n_neighbors (int): n_neighbors for UMAP.
        min_dist (float): min_dist for UMAP.
        title_suffix (str): Optional suffix for the plot title.
    """
    if layer_name not in all_activations or not all_activations[layer_name]:
        print(f"No activations found for layer {layer_name}")
        return


    activations_np = np.array(all_activations[layer_name]).squeeze()


    print(f"Projecting activations for layer: {layer_name} (Shape: {activations_np.shape}) using {method.upper()}...")

    if method == 'tsne':
        reducer = TSNE(n_components=2, random_state=42, perplexity=min(perplexity, activations_np.shape[0] - 1)) # Perplexity must be less than n_samples
    elif method == 'umap':
        try:
             from umap import UMAP # Check import locally
             reducer = UMAP(n_components=2, random_state=42, n_neighbors=min(n_neighbors, activations_np.shape[0] - 1), min_dist=min_dist)
        except ImportError:
             print("UMAP not installed. Install umap-learn. Falling back to t-SNE.")
             reducer = TSNE(n_components=2, random_state=42, perplexity=min(perplexity, activations_np.shape[0] - 1))
    else:
        raise ValueError("Method must be 'tsne' or 'umap'")

    projections = reducer.fit_transform(activations_np)

    plt.figure(figsize=(8, 6))
    sns.scatterplot(x=projections[:, 0], y=projections[:, 1], hue=labels, palette='viridis', s=50, alpha=0.8)
    plt.title(f'{method.upper()} Projection of Activations\nLayer: {layer_name}{title_suffix}')
    plt.xlabel(f'{method.upper()} Dimension 1')
    plt.ylabel(f'{method.upper()} Dimension 2')
    plt.legend(title='Input Type')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()


def plot_steering_vector(steering_vector, layer_name, k=20):
    """
    Plots the steering vector values and a bar chart of the top K dimensions by magnitude.

    Args:
        steering_vector (np.ndarray): The calculated steering vector (1D).
        layer_name (str): The layer name for the title.
        k (int): Number of top dimensions to show in the bar chart.
    """
    if steering_vector is None or steering_vector.size == 0:
        print(f"Invalid steering vector for layer {layer_name}")
        return

    steering_vector = steering_vector.flatten() # Ensure 1D

    fig, axs = plt.subplots(1, 2, figsize=(15, 5))
    fig.suptitle(f"Steering Vector Analysis - Layer: {layer_name}", fontsize=14)

    # Plot 1: Line plot of the vector values
    axs[0].plot(steering_vector)
    axs[0].set_title('Steering Vector Values')
    axs[0].set_xlabel('Dimension Index')
    axs[0].set_ylabel('Value')
    axs[0].grid(True, linestyle='--', alpha=0.6)

    # Plot 2: Top K dimensions by absolute value
    if k > 0 and steering_vector.size > 0:
         k = min(k, steering_vector.size) # Adjust k if vector is smaller
         top_k_indices = np.argsort(np.abs(steering_vector))[-k:]
         top_k_values = steering_vector[top_k_indices]
         dim_labels = [f'Dim {i}' for i in top_k_indices]

         colors = ['red' if v < 0 else 'blue' for v in top_k_values]
         axs[1].bar(range(k), top_k_values, color=colors)
         axs[1].set_xticks(range(k))
         axs[1].set_xticklabels(dim_labels, rotation=90)
         axs[1].set_title(f'Top {k} Dimensions by Magnitude')
         axs[1].set_ylabel('Value')
         axs[1].grid(True, axis='y', linestyle='--', alpha=0.6)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust layout to prevent title overlap
    plt.show()


def plot_steering_vector_norms(average_diff_activations):
    """
    Plots the L2 norm of steering vectors across different layers.

    Args:
        average_diff_activations (dict): Dictionary from layer_name to steering vector (np.ndarray).
    """
    if not average_diff_activations:
        print("No steering vectors provided.")
        return

    layer_names = list(average_diff_activations.keys())
    norms = [np.linalg.norm(vec) for vec in average_diff_activations.values() if vec is not None]
    valid_layer_names = [name for name, vec in average_diff_activations.items() if vec is not None]

    if not norms:
        print("No valid steering vectors found to calculate norms.")
        return

    plt.figure(figsize=(12, 6))
    plt.bar(range(len(norms)), norms, tick_label=valid_layer_names)
    plt.xticks(rotation=90)
    plt.ylabel('L2 Norm (Magnitude)')
    plt.title('Steering Vector Norms Across Layers')
    plt.grid(True, axis='y', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()


def plot_steering_vector_similarity(average_diff_activations):
    """
    Plots a heatmap of the cosine similarity between steering vectors from different layers.

    Args:
        average_diff_activations (dict): Dictionary from layer_name to steering vector (np.ndarray).
    """
    if not average_diff_activations:
        print("No steering vectors provided.")
        return

    layer_names = list(average_diff_activations.keys())
    vectors = [vec.flatten() for vec in average_diff_activations.values() if vec is not None]
    valid_layer_names = [name for name, vec in average_diff_activations.items() if vec is not None]

    if len(vectors) < 2:
        print("Need at least two valid steering vectors to compare similarity.")
        return

    # Ensure all vectors are 1D for cosine similarity calculation if needed,
    # although cosine_similarity handles 2D arrays row-wise. Stacking is safer.
    try:
         vector_matrix = np.stack(vectors) # Shape (n_layers, hidden_dim)
    except ValueError as e:
         print(f"Error stacking vectors, likely due to inconsistent dimensions: {e}")
         # Attempt to pad or resize if necessary, or just report error
         max_len = max(v.shape[0] for v in vectors)
         padded_vectors = []
         for v in vectors:
             if v.shape[0] < max_len:
                 pad_width = max_len - v.shape[0]
                 padded_v = np.pad(v, (0, pad_width), 'constant')
                 padded_vectors.append(padded_v)
                 print(f"Warning: Padded vector for layer {valid_layer_names[len(padded_vectors)-1]}")
             else:
                 padded_vectors.append(v)
         vector_matrix = np.stack(padded_vectors)
         # return # Alternatively, decide whether to proceed with potentially padded vectors


    similarity_matrix = cosine_similarity(vector_matrix)

    plt.figure(figsize=(10, 8))
    sns.heatmap(similarity_matrix, annot=True, cmap='coolwarm', fmt=".2f",
                xticklabels=valid_layer_names, yticklabels=valid_layer_names)
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    plt.title('Cosine Similarity Between Steering Vectors Across Layers')
    plt.tight_layout()
    plt.show()


def compare_generation(steerer, prompts, steering_params, max_length=50):
    """
    Generates text with and without steering for comparison.

    Args:
        steerer (ActivationController): An initialized ActivationController instance.
        prompts (list): A list of input prompts.
        steering_params (dict): A dict mapping layer_name to {'vector': np.ndarray, 'coeff': float}.
                                The steering vector should be a numpy array.
        max_length (int): Max new tokens to generate.

    Returns:
        pandas.DataFrame: DataFrame with prompts, unsteered, and steered outputs.
    """
    results = []
    device = steerer.device

    # Prepare steering vectors on the correct device
    steering_vectors_gpu = {}
    for layer_name, params in steering_params.items():
        vector_np = params['vector']
        # Ensure vector is compatible shape for broadcasting (e.g., (1, 1, hidden_dim))
        # This depends on the activation shape inside the hook. Let's assume the hook
        # expects a transformation on an activation of shape (batch, seq, hidden).
        # If vector_np is (hidden_dim,), reshape it.
        if vector_np.ndim == 1:
             # Reshape for broadcasting: (1, 1, hidden_dim) might be needed
             # Or let the hook handle it if it's simple addition.
             # For safety, let's add batch and sequence dimensions.
             vector_reshaped = vector_np.reshape(1, 1, -1)
        else:
             # Assume it's already correctly shaped if not 1D
             vector_reshaped = vector_np

        steering_vectors_gpu[layer_name] = torch.tensor(vector_reshaped, dtype=torch.float16 if 'cuda' in device else torch.float32).to(device) # Use float16 on GPU


    for prompt in prompts:
        print(f"\nProcessing prompt: '{prompt}'")

        # --- Unsteered Generation ---
        print("  Generating (Unsteered)...")
        steerer.clear_hooks() # Ensure no leftover transformations
        text_unsteered = steerer.generate(prompt, max_length=max_length)
        print(f"    Unsteered: {text_unsteered}")

        # --- Steered Generation ---
        print("  Generating (Steered)...")
        steerer.clear_hooks() # Clear just in case, then set new ones
        for layer_name, params in steering_params.items():
            if layer_name in steerer.hooks:
                coeff = params['coeff']
                vector_gpu = steering_vectors_gpu[layer_name]

                # Define the transformation function using the pre-moved tensor
                # This lambda will be called by the hook with the activation tensor
                def transformation_func(activation_tensor, vec=vector_gpu, c=coeff):
                    # Add the steering vector (broadcasting should handle dimensions)
                    # Ensure dtype compatibility if using AMP
                    return activation_tensor.to(vec.dtype) + c * vec

                steerer.set_transformation_function(layer_name, transformation_func)
                print(f"    Applied steering to {layer_name} with coeff {coeff}")
            else:
                print(f"    Warning: Layer {layer_name} not found in hooks.")

        text_steered = steerer.generate(prompt, max_length=max_length)
        print(f"    Steered:   {text_steered}")

        results.append({
            "Prompt": prompt,
            "Unsteered Output": text_unsteered,
            "Steered Output": text_steered
        })

        # Clear hooks for the next prompt
        steerer.clear_hooks()

    return pd.DataFrame(results)


# --- Example Usage ---

if __name__ == '__main__':
    # This is placeholder execution logic. Replace with your actual model loading
    # and data preparation. Assume you have:
    # - `model_name`: Name of the HF model
    # - `positive_texts`, `negative_texts`: Lists of strings for extracting vectors
    # - `test_prompts`: List of strings to test generation steering

    print("Setting up steerers (Example - replace with your model)...")
    MODEL_NAME = "unsloth/Llama-3.3-70B-Instruct" # Example - use a model you have access to

    selected_blocks = ['model.layers.30.self_attn', 'model.layers.30.mlp']
    avg_steerer = ActAddSteerer(MODEL_NAME, selected_blocks=selected_blocks) # Uses LLMSteerer internally



    # --- 1. Extract Activations (Example Data) ---
    indices = np.random.choice(len(negative_essay_prompts), size=100, replace=False)
    positive_texts = [encourageing_essay_prompts[i] for i in indices] # Using list comprehension for proper indexing
    negative_texts = [negative_essay_prompts[i] for i in indices] # Same fix for negative_texts
    all_texts = positive_texts + negative_texts
    labels = ['positive'] * len(positive_texts) + ['negative'] * len(negative_texts)

    all_activations = avg_steerer.extract_activations(all_texts, aggregation_calc="other")

    target_layer = avg_steerer.layer_names[len(avg_steerer.layer_names)-1] # Example: middle layer ml

    plot_activation_projection(all_activations, target_layer, labels, method='tsne')

    all_activations = None # Clear to free up memory

    # --- 3. Calculate and Visualize Steering Vector ---
    # Use default aggregation (last token) for steering vector calculation
    average_diff_activations = avg_steerer.extract_average_diff(positive_texts, negative_texts)

    plot_steering_vector(average_diff_activations[target_layer], target_layer, k=20)

    print("\nPlotting steering vector norms...")
    plot_steering_vector_norms(average_diff_activations)

    print("\nPlotting steering vector similarity...")
    plot_steering_vector_similarity(average_diff_activations)


    # --- 4. Compare Generation ---
    print("\nComparing steered vs. unsteered generation...")
    test_prompts = [
        "The weather today is",
        "I thought the presentation was",
        "This policy change seems",
        "User Input: I really like strawberries.\nAI Response: Strawberries are", # Example for steering tone
        "User Input: I really hate spiders.\nAI Response: Spiders are"
    ]

    # Define steering parameters (apply the positive vector with a coefficient)
    # Apply to one or more layers where the vector norm was significant
    steering_params = {}
    steering_coeff = 10 # Example coefficient - TUNE THIS!
    if target_layer in average_diff_activations and average_diff_activations[target_layer] is not None:
        steering_params[target_layer] = {
            'vector': average_diff_activations[target_layer], # The pos-neg vector
            'coeff': steering_coeff
        }

    comparison_df = compare_generation(avg_steerer, test_prompts, steering_params, max_length=20)


In [ ]:
# Load up model and get example
model = "unsloth/Llama-3.2-3B"
activation_steerer = ActAddSteerer(model)
output1 = activation_steerer.generate(negative_essay_prompts[5], max_length=250)
print(output1)

# Load up the diffs
indices = np.random.choice(len(negative_essay_prompts), size=10, replace=False)
rand_good = encourageing_essay_prompts[indices]
rand_negative = negative_essay_prompts[indices]

average_diff = activation_steerer.extract_average_diff(rand_good, rand_negative)

In [ ]:
for layer_name in activation_steerer.layer_names:
    print(layer_name)
    added_tensor = torch.tensor(average_diff[layer_name]).to(activation_steerer.model.device)
    def add_to_last_index(x, added_tensor=added_tensor):
        x[:, -1, :] += added_tensor
        return x

    activation_steerer.set_transformation_function(layer_name, add_to_last_index)

print(activation_steerer.generate(negative_essay_prompts[5], max_length=250))

for layer_name in activation_steerer.layer_names:
    print(layer_name)

    activation_steerer.clear_hooks()
    added_tensor = torch.tensor(average_diff[layer_name]).to(activation_steerer.model.device)
    if int(layer_name.split('.')[2]) == 15:
        added_tensor = added_tensor * 1000
    def add_to_last_index(x, added_tensor=added_tensor):
        x[:, -1, :] += added_tensor
        return x

    activation_steerer.set_transformation_function(layer_name, add_to_last_index)
    print(activation_steerer.generate(negative_essay_prompts[5], max_length=100))


In [ ]:
import gc

gc.collect()
torch.cuda.empty_cache()


In [ ]:
!pip install langchain langchain_anthropic langchain_openai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.1/420.1 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 60.5 MB/s eta 0:00:00


In [ ]:
import os
import json
from typing import Dict, List, Any
from langchain.llms import BaseLLM
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage


class FeedbackEvaluator:
    """
    A model-agnostic class to evaluate feedback using LangChain with various LLM providers.
    """

    def __init__(self, provider: str = "anthropic", model_name: str = None):
        """
        Initialize the FeedbackEvaluator with a specific LLM provider.

        Args:
            provider: The LLM provider to use ('anthropic', 'openai', etc.)
            model_name: The specific model to use (optional, will use default if not specified)
        """
        self.provider = provider
        self.model_name = model_name
        self.llm = self._initialize_llm()

    def _initialize_llm(self) -> BaseLLM:
        """
        Initialize the LLM based on the specified provider.

        Returns:
            A LangChain LLM instance
        """
        if self.provider == "anthropic":
            model = self.model_name or "claude-3-5-sonnet-20240620"
            return ChatAnthropic(
                anthropic_api_key=ANTHROPIC_KEY,
                model=model,
                temperature=0.2,
                max_tokens=4000
            )
        elif self.provider == "openai":
            model = self.model_name or "gpt-4"
            return ChatOpenAI(
                openai_api_key=os.environ.get("OPENAI_API_KEY"),
                model=model,
                temperature=0.2,
                max_tokens=4000
            )
        else:
            raise ValueError(f"Unsupported provider: {self.provider}")

    def _call_llm(self, prompt: str, system_prompt: str) -> str:
        """
        Call the LLM with a prompt and return the response.

        Args:
            prompt: The prompt to send to the LLM
            system_prompt: The system prompt to use

        Returns:
            The LLM's response
        """
        try:
            messages = [
                SystemMessage(content=system_prompt),
                HumanMessage(content=prompt)
            ]
            response = self.llm.invoke(messages)
            return response.content
        except Exception as e:
            print(f"Error calling LLM: {e}")
            return ""

    def evaluate_macfarlane_dick(self, feedback: str) -> Dict[str, Any]:
        """
        Evaluate feedback based on Nicol & Macfarlane-Dick's seven principles of good feedback.

        Args:
            feedback: The feedback text to evaluate

        Returns:
            A dictionary containing the evaluation results
        """
        system_prompt = """You are an expert in educational assessment and feedback evaluation.
                Your task is to analyze feedback objectively and provide structured evaluations.
                Provide your analysis in JSON format as specified in the prompt."""

        prompt = f"""
        Evaluate the following feedback according to Nicol & Macfarlane-Dick's seven principles of good feedback practice:

        1. Helps clarify what good performance is
        2. Facilitates the development of self-assessment
        3. Delivers high-quality information to students about their learning
        4. Encourages teacher and peer dialogue around learning
        5. Encourages positive motivational beliefs and self-esteem
        6. Provides opportunities to close the gap between current and desired performance
        7. Provides information to teachers that can be used to help shape teaching

        Feedback to evaluate:
        "{feedback}"

        For each principle, provide:
        1. A score from 0 to 5 (where 0 is not at all and 5 is exemplary)
        2. A brief explanation of why you gave this score
        3. A suggestion for improvement

        Return your evaluation as a JSON object with the following structure:
        {{
            "overall_score": <average of all scores, rounded to one decimal place>,
            "principles": [
                {{
                    "principle": "Principle 1",
                    "score": <score>,
                    "explanation": "<explanation>",
                    "suggestion": "<suggestion>"
                }},
                ...
            ],
            "strengths": ["<strength 1>", "<strength 2>", ...],
            "areas_for_improvement": ["<area 1>", "<area 2>", ...],
            "summary": "<brief summary of evaluation>"
        }}
        """

        response = self._call_llm(prompt, system_prompt)
        try:
            return json.loads(response)
        except json.JSONDecodeError:
            print("Failed to parse JSON response from LLM")
            return {
                "error": "Failed to parse evaluation",
                "raw_response": response
            }

    def compare_feedback(self, feedback1: str, feedback2: str) -> Dict[str, Any]:
        """
        Compare two pieces of feedback and determine which is better.

        Args:
            feedback1: The first feedback text
            feedback2: The second feedback text

        Returns:
            A dictionary containing the comparison results
        """
        system_prompt = """You are an expert in educational assessment and feedback evaluation.
                Your task is to analyze and compare feedback objectively.
                Provide your analysis in JSON format as specified in the prompt."""

        prompt = f"""
        Compare the following two pieces of feedback and determine which one is more effective:

        Feedback 1:
        "{feedback1}"

        Feedback 2:
        "{feedback2}"

        Consider the following aspects in your comparison:
        1. Clarity and specificity
        2. Actionability
        3. Balance of positive and constructive elements
        4. Alignment with learning goals
        5. Potential impact on student motivation and learning

        Return your comparison as a JSON object with the following structure:
        {{
            "winner": <1 or 2, indicating which feedback is better>,
            "score_difference": <a number from 0 to 10 indicating how much better the winner is>,
            "aspects": [
                {{
                    "aspect": "Clarity and specificity",
                    "feedback1_score": <score from 0 to 5>,
                    "feedback2_score": <score from 0 to 5>,
                    "explanation": "<explanation>"
                }},
                ...
            ],
            "feedback1_strengths": ["<strength 1>", ...],
            "feedback1_weaknesses": ["<weakness 1>", ...],
            "feedback2_strengths": ["<strength 1>", ...],
            "feedback2_weaknesses": ["<weakness 1>", ...],
            "recommendation": "<recommendation for how the lower-rated feedback could be improved>"
        }}
        """

        response = self._call_llm(prompt, system_prompt)
        try:
            return json.loads(response)
        except json.JSONDecodeError:
            print("Failed to parse JSON response from LLM")
            return {
                "error": "Failed to parse comparison",
                "raw_response": response
            }

    def evaluate_kindness(self, feedback: str) -> Dict[str, Any]:
        """
        Evaluate the kindness of feedback on a scale from 0 to 5.

        Args:
            feedback: The feedback text to evaluate

        Returns:
            A dictionary containing the kindness evaluation
        """
        system_prompt = """You are an expert in educational assessment and feedback evaluation.
                Your task is to analyze the kindness and empathy in feedback.
                Provide your analysis in JSON format as specified in the prompt."""

        prompt = f"""
        Evaluate the kindness of the following feedback on a scale from 0 to 5, where:

        0: Harsh, potentially harmful or demotivating
        1: Cold and impersonal
        2: Neutral
        3: Somewhat kind and supportive
        4: Kind and encouraging
        5: Exceptionally kind, empathetic and motivating

        Feedback to evaluate:
        "{feedback}"

        Consider the following aspects of kindness in feedback:
        1. Tone and language choice
        2. Balance of positive and constructive elements
        3. Recognition of effort and achievement
        4. Personalization
        5. Empathy and understanding

        Return your evaluation as a JSON object with the following structure:
        {{
            "kindness_score": <score from 0 to 5>,
            "explanation": "<explanation for the score>",
            "aspects": [
                {{
                    "aspect": "Tone and language choice",
                    "score": <score from 0 to 5>,
                    "explanation": "<explanation>"
                }},
                ...
            ],
            "examples": ["<example of kind/unkind language in the feedback>", ...],
            "improvement_suggestions": ["<suggestion 1>", ...],
            "summary": "<brief summary of kindness evaluation>"
        }}
        """

        response = self._call_llm(prompt, system_prompt)
        try:
            return json.loads(response)
        except json.JSONDecodeError:
            print("Failed to parse JSON response from LLM")
            return {
                "error": "Failed to parse kindness evaluation",
                "raw_response": response
            }

# Example usage
if __name__ == "__main__":
    # Example feedback
    sample_feedback = """
    Your essay shows good understanding of the topic, but your arguments need more support.
    Try to include more specific examples and cite relevant sources to strengthen your points.
    I liked your introduction, it was clear and engaging. Your conclusion could be stronger
    by restating your main points and providing a thoughtful closing statement.
    """

    # Create an evaluator
    evaluator = FeedbackEvaluator(provider="anthropic")

    # Example 1: Evaluate using Macfarlane-Dick principles
    print("\n=== Macfarlane-Dick Evaluation ===")
    result = evaluator.evaluate_macfarlane_dick(sample_feedback)
    print(json.dumps(result, indent=2))

    # Example 2: Compare two pieces of feedback
    sample_feedback2 = """
    This essay needs a lot of work. The arguments are weak and poorly supported.
    Your writing style is confusing, and there are many grammar errors throughout.
    You should completely revise this and try again.
    """

    print("\n=== Feedback Comparison ===")
    result = evaluator.compare_feedback(sample_feedback, sample_feedback2)
    print(json.dumps(result, indent=2))

    # Example 3: Evaluate kindness
    print("\n=== Kindness Evaluation ===")
    result = evaluator.evaluate_kindness(sample_feedback)
    print(json.dumps(result, indent=2))


=== Macfarlane-Dick Evaluation ===
{
  "overall_score": 3.3,
  "principles": [
    {
      "principle": "Helps clarify what good performance is",
      "score": 3,
      "explanation": "The feedback provides some guidance on what constitutes good performance, such as including specific examples and citing sources, but it could be more comprehensive.",
      "suggestion": "Include more specific criteria for what makes a strong essay, such as clear thesis statement, logical structure, and effective use of evidence."
    },
    {
      "principle": "Facilitates the development of self-assessment",
      "score": 2,
      "explanation": "The feedback doesn't explicitly encourage self-assessment, though it does point out areas for improvement which could prompt reflection.",
      "suggestion": "Include questions that encourage the student to reflect on their work, such as 'How might you strengthen your arguments further?'"
    },
    {
      "principle": "Delivers high-quality information

# FGSM Technique

In [ ]:
class LinearProbeSteerer(ActivationController):
    def __init__(self, model, selected_blocks=None):
        super().__init__(model, selected_blocks=selected_blocks)

        self.best_layer = None
        self.best_model = None
        self.classifier = None

    def train_classifier(self, positive_texts, negative_texts, device='cuda'):
        positive_activations = self.extract_activations(positive_texts, device)
        negative_activations = self.extract_activations(negative_texts, device)

        results = {}
        best_accuracy = 0
        best_layer = ""
        best_model = None
        for layer_name in positive_activations:
            positive_examples = np.array(positive_activations[layer_name])
            negative_examples = np.array(negative_activations[layer_name])
            print("Positive Examples Lne", len(positive_examples))
            print("Negative Examples Lne", len(negative_examples))

            labels = [1] * len(positive_examples) + [0] * len(positive_examples)

            all_activations = list(positive_examples) + list(negative_examples)
            all_activations = np.concatenate(all_activations, axis=0)

            X_train, X_test, y_train, y_test = train_test_split(
                all_activations, labels, test_size=0.3, random_state=42
            )

            # Train classifier
            self.classifier.fit(X_train, y_train)
            print("Y Train", y_train[:10])

            y_pred = self.classifier.predict(X_test)
            print("y_pred", y_pred[:5])
            print("X_test", X_test[:5])
            report = classification_report(y_test, y_pred, output_dict=True)
            results[layer_name] = report['accuracy']

            print(f"Accuracy: {report['accuracy']:.4f}")
            print(classification_report(y_test, y_pred))

            if report['accuracy'] > best_accuracy:
                best_accuracy = report['accuracy']
                best_layer = layer_name
                best_model = self.classifier

        # Visualize results
        plt.figure(figsize=(10, 6))
        layers = list(results.keys())
        accuracies = [results[layer] for layer in layers]

        plt.bar(range(len(results)), accuracies)
        plt.xlabel('Layer')
        plt.ylabel('Accuracy')
        plt.title('Classification Accuracy by Layer')
        plt.xticks(range(len(results)), [f"Layer {layer.split('.')[-2]} {layer.split('.')[-1]}" for layer in layers])
        plt.tight_layout()
        plt.savefig('steering_output/layer_accuracies.png')
        print("Results visualization saved to 'steering_output/layer_accuracies.png'")

        print("Best model", best_model.coef_)
        self.best_model = best_model
        self.best_layer = best_layer
        self.classifier = best_model
        return best_layer, best_model

    def set_transformation_function(self, c=1, func_type='multiply'):
        vector_gpu = torch.tensor(self.classifier.coef_[0], dtype=torch.float32)
        vector_gpu = vector_gpu.to(activation_steerer.device)

        def mult_func(activation_tensor, vec=vector_gpu, c=c):
            return activation_tensor * (1 + (vec.to(activation_tensor.dtype ) * c))

        def add_func(activation_tensor, vec=vector_gpu, c=c):
            return activation_tensor + (vec.to(activation_tensor.dtype ) * c)

        if func_type == "add":
            return super().set_transformation_function(self.layer_name, add_func)
        elif func_type == "multiply":
            return super().set_transformation_function(self.layer_name, mult_func)
        else:
            raise ValueError(f"Unsupported function type: {func_type}")

In [ ]:
import copy
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import classification_report, accuracy_score

class SimpleMLPProbe(nn.Module):
    """A simple linear probe for binary classification."""
    def __init__(self, input_dim, hidden_dim=256, output_dim=256, dropout_prob=0.1):
        super().__init__()
        # Linear layer mapping activation dimension to a single logit
        self.linear = nn.Sequential(
                nn.Linear(input_dim, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, output_dim),
                nn.ReLU(),
                nn.Linear(output_dim, 1),
                nn.Sigmoid()
            )

    def forward(self, x):
        # Output raw logits (suitable for BCEWithLogitsLoss)
        return self.linear(x)
    

class TorchModelSteerer(ActivationController):
    def __init__(self, model, selected_blocks=None,
                 # Probe & Training Hyperparameters
                 learning_rate=0.001, epochs=10, batch_size=32,
                 probe_type='mlp',
                 mlp_hidden_dim=None, mlp_dropout_prob=0.1):
        super().__init__(model, selected_blocks=selected_blocks)

        self.best_layer = None
        self.best_model_state_dict = None
        self.best_model_input_dim = None
        self.best_probe_type = None # Store the type of the best probe

        # Store hyperparameters
        self.lr = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        self.probe_type = probe_type
        self.mlp_hidden_dim = mlp_hidden_dim
        self.mlp_dropout_prob = mlp_dropout_prob

    def train_classifier(self, positive_texts, negative_texts, device='cuda'):
        # Ensure the target device is a torch device
        torch_device = torch.device(device if torch.cuda.is_available() else "cpu")
        print(f"Using device: {torch_device}")

        # Extract activations (assuming this returns a dict layer_name -> list of numpy arrays)
        # Make sure activations are extracted correctly (e.g., mean pooled per text)
        positive_activations = self.extract_activations(positive_texts, device) # Keep extraction device potentially different
        negative_activations = self.extract_activations(negative_texts, device)

        results = {}
        best_accuracy = 0
        best_layer = ""
        best_model_state_dict = None
        best_model_input_dim = None

        for layer_name in positive_activations:
            # Ensure we have activations for this layer from both sets
            if layer_name not in negative_activations or \
               len(positive_activations[layer_name]) == 0 or \
               len(negative_activations[layer_name]) == 0:
                print(f"Skipping layer {layer_name}: Missing activations.")
                continue

            # It's crucial that each item in the list is ONE vector per text input
            # If extract_activations returns activations per token, you need to aggregate first (e.g., mean)
            try:
                # Assuming list contains numpy arrays ready to be stacked
                positive_examples = np.vstack(positive_activations[layer_name])
                negative_examples = np.vstack(negative_activations[layer_name])
            except ValueError as e:
                print(f"Error stacking activations for layer {layer_name}: {e}")
                print(f"Shapes: Pos {[a.shape for a in positive_activations[layer_name]]}, Neg {[a.shape for a in negative_activations[layer_name]]}")
                continue # Skip layer if shapes mismatch

            print(f"\n--- Training Layer: {layer_name} ---")
            print(f"Positive Examples shape: {positive_examples.shape}")
            print(f"Negative Examples shape: {negative_examples.shape}")


            activation_dim = positive_examples.shape[1]

            # --- Data Preparation ---
            # Correct label creation
            labels_pos = np.ones(len(positive_examples), dtype=np.float32)
            labels_neg = np.zeros(len(negative_examples), dtype=np.float32)

            all_activations_np = np.vstack((positive_examples, negative_examples))
            all_labels_np = np.concatenate((labels_pos, labels_neg))

            # Split data
            X_train_np, X_test_np, y_train_np, y_test_np = train_test_split(
                all_activations_np, all_labels_np, test_size=0.3, random_state=42, stratify=all_labels_np
            )

            # Convert to PyTorch Tensors
            X_train = torch.tensor(X_train_np, dtype=torch.float32)
            y_train = torch.tensor(y_train_np, dtype=torch.float32).unsqueeze(1) # Add dim for BCEWithLogitsLoss
            X_test = torch.tensor(X_test_np, dtype=torch.float32)
            y_test = torch.tensor(y_test_np, dtype=torch.float32).unsqueeze(1)

            # Create DataLoaders
            train_dataset = TensorDataset(X_train, y_train)
            test_dataset = TensorDataset(X_test, y_test)
            train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
            test_loader = DataLoader(test_dataset, batch_size=self.batch_size, shuffle=False)

            # --- Model, Loss, Optimizer ---
            probe_model = SimpleMLPProbe(activation_dim).to(torch_device)
            criterion = nn.BCEWithLogitsLoss() # Numerically stable
            optimizer = optim.Adam(probe_model.parameters(), lr=self.lr)

            # --- Training Loop ---
            probe_model.train()
            for epoch in range(self.epochs):
                epoch_loss = 0
                for batch_X, batch_y in train_loader:
                    batch_X, batch_y = batch_X.to(torch_device), batch_y.to(torch_device)

                    optimizer.zero_grad()
                    outputs = probe_model(batch_X)
                    loss = criterion(outputs, batch_y)
                    loss.backward()
                    optimizer.step()
                    epoch_loss += loss.item()
                # print(f"Epoch [{epoch+1}/{self.epochs}], Loss: {epoch_loss/len(train_loader):.4f}")

            # --- Evaluation ---
            probe_model.eval()
            all_preds = []
            all_targets = []
            with torch.no_grad():
                for batch_X, batch_y in test_loader:
                    batch_X = batch_X.to(torch_device)
                    outputs = probe_model(batch_X)
                    # Convert logits to probabilities then to predictions (0 or 1)
                    preds = (torch.sigmoid(outputs) > 0.5).cpu().long().squeeze().numpy()
                    all_preds.extend(preds.tolist())
                    all_targets.extend(batch_y.cpu().long().squeeze().numpy().tolist())

            # Calculate accuracy and report
            # Ensure consistent types if using lists directly
            try:
                accuracy = accuracy_score(all_targets, all_preds)
                report_str = classification_report(all_targets, all_preds)
                print(f"Accuracy: {accuracy:.4f}")
                print(report_str)
                results[layer_name] = accuracy

                # --- Update Best Model ---
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_layer = layer_name
                    # Save the state_dict and input dim
                    best_model_state_dict = copy.deepcopy(probe_model.state_dict())
                    best_model_input_dim = activation_dim
                    print(f"*** New best layer: {best_layer} (Accuracy: {best_accuracy:.4f}) ***")
            except Exception as e:
                print(f"Error during evaluation for layer {layer_name}: {e}")
                results[layer_name] = 0.0 # Assign low score if evaluation fails


        # --- Visualization (remains the same) ---
        if results:
          plt.figure(figsize=(10, 6))
          layers = list(results.keys())
          accuracies = [results[layer] for layer in layers]

          plt.bar(range(len(results)), accuracies)
          plt.xlabel('Layer')
          plt.ylabel('Accuracy')
          plt.title('Classification Accuracy by Layer (PyTorch Probe)')
          # Adjust labels if needed, e.g., rotate
          plt.xticks(range(len(results)), [f"L {'.'.join(layer.split('.')[-2:])}" for layer in layers], rotation=45, ha='right')
          plt.tight_layout()
          plt.savefig('steering_output/layer_accuracies_pytorch.png')
          print("Results visualization saved to 'steering_output/layer_accuracies_pytorch.png'")
        else:
          print("No results to plot.")


        # Store best model info in the instance
        self.best_model_state_dict = best_model_state_dict
        self.best_model_input_dim = best_model_input_dim
        self.best_layer = best_layer

        if self.best_layer:
            print(f"\nBest overall layer: {self.best_layer} with accuracy: {best_accuracy:.4f}")
        else:
            print("\nFailed to find a suitable layer for steering.")

        # Return layer name and best accuracy (or model info if needed)
        return self.best_layer, best_accuracy


    
    @torch.no_grad() # Disable gradients for the main generation loop
    def generate(self, tokenizer, prompt,
                 max_length=50, pnp_step_size=0.02, pnp_target_label=1.0,
                 temperature=1.0, top_k=50, device=None):

        """Generates text using PnP steering with the trained probe."""


        if device is None:
            device = self.model.device if hasattr(self.model, 'device') else \
                     torch.device("cuda" if torch.cuda.is_available() else "cpu")


        probe_model = SimpleMLPProbe(
            input_dim=self.best_model_input_dim,
            hidden_dim=self.mlp_hidden_dim, # Relies on stored param
            dropout_prob=self.mlp_dropout_prob # Relies on stored param
        ).to(device)
        probe_model.load_state_dict(self.best_model_state_dict)
        probe_model.eval()

        # 2. Prepare inputs
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=self.model.config.max_position_embeddings - max_length).to(device)
        input_ids = inputs.input_ids
        attention_mask = inputs.attention_mask
        past_key_values = None
        generated_ids = input_ids.clone()

        # --- 4. Generation Loop ---
        try:
            for step in range(max_length):
                
                activations = self.extract_activations(prompt, device)


                # --- Forward pass (captures state via hook) ---
                # Enable gradients temporarily ONLY for PnP update if needed below
                outputs = self.model(**model_inputs, output_hidden_states=False, use_cache=True)
                next_token_logits = outputs.logits[:, -1, :] # Logits for the last token
                past_key_values = outputs.past_key_values

                # --- PnP Update Step ---
                h_t = captured_state_dict['value'] # State from hook
                if h_t is not None:
                    # Enable gradient calculation for this part
                    with torch.enable_grad():
                        h_t = h_t.requires_grad_(True)

                        # Forward through probe
                        logits_probe = probe_model(h_t)
                        criterion = nn.BCEWithLogitsLoss()
                        # Ensure target label is float and on correct device
                        target = torch.full_like(logits_probe, float(pnp_target_label), device=device)
                        loss = criterion(logits_probe, target)

                        # Backward pass to get gradient w.r.t. h_t
                        loss.backward()
                        grad = h_t.grad

                    # --- Update hidden state ---
                    if grad is not None:
                        norm_grad = grad / (grad.norm(p=2, dim=-1, keepdim=True) + 1e-8)
                        # Update happens outside grad context
                        h_t_updated = h_t.detach() - pnp_step_size * norm_grad

                        # --- Use updated state to modify logits (Model Dependent!) ---
                        # This is the hardest part to generalize.
                        # Option A: Rerun LM Head (if accessible & state shape matches)
                        try:
                            # Check if lm_head expects [batch, seq, hidden] or [batch, hidden]
                            if hasattr(self.model, 'lm_head') and callable(self.model.lm_head):
                                if h_t_updated.dim() == 2: # [batch, hidden] -> [batch, 1, hidden]
                                   h_t_updated_seq = h_t_updated.unsqueeze(1)
                                elif h_t_updated.dim() == 3: # Assume shape is already ok
                                   h_t_updated_seq = h_t_updated
                                else: raise ValueError("Unexpected state dimension")

                                # Recalculate logits for the last position
                                next_token_logits = self.model.lm_head(h_t_updated_seq).squeeze(1) # [batch, vocab]
                                print(f"Step {step}: Recalculated logits.") # Debug print
                            else:
                                print(f"Step {step}: Cannot access lm_head directly, skipping logit recalculation.")

                        except Exception as e:
                             print(f"Step {step}: Error recalculating logits with updated state: {e}. Using original logits.")

                        # Option B: Perturb original logits (Approximation)
                        # Requires calculating grad w.r.t logits or using W_lm * grad approx.
                        # (Not implemented here for simplicity)

                        # --- Clean Gradients (on original h_t) ---
                        # Not strictly needed as h_t is recreated each loop, but good practice
                        # if h_t had persistent graph connections.

                    # Reset captured state for next iteration
                    captured_state_dict['value'] = None
                else:
                     print(f"Warning: Step {step}: Did not capture hidden state.")

                # --- Token Sampling (outside grad context) ---
                if temperature != 1.0:
                    next_token_logits = next_token_logits / temperature

                if top_k > 0:
                    # Slightly safer top-k implementation
                    indices_to_remove = next_token_logits < torch.topk(next_token_logits, top_k)[0][..., -1, None]
                    next_token_logits[indices_to_remove] = -float('Inf')

                probs = F.softmax(next_token_logits, dim=-1)
                next_token_id = torch.multinomial(probs, num_samples=1)

                # --- Update generated sequence ---
                generated_ids = torch.cat([generated_ids, next_token_id], dim=-1)
                if attention_mask is not None:
                    # Pad attention mask if needed by model.prepare_inputs
                    attention_mask = torch.cat([attention_mask, torch.ones_like(next_token_id)], dim=1)

                # --- Check for stopping ---
                if tokenizer.eos_token_id is not None and (next_token_id == tokenizer.eos_token_id).all():
                    print("EOS token generated.")
                    break
        except Exception as e:
            print(f"Error during generation loop: {e}")
            # Ensure hook removal even if loop fails

        # 6. Decode result
        return tokenizer.decode(generated_ids[0], skip_special_tokens=True)

In [ ]:
vector_gpu = torch.tensor(activation_steerer.classifier.coef_[0], dtype=torch.float32)
vector_gpu = vector_gpu.to(activation_steerer.device)


activation_steerer.clear_hooks()
prompt = encourageing_essay_prompts[0]
max_length = 25
print(activation_steerer.generate(prompt, max_length=max_length))
print(activation_steerer.generate(prompt, max_length=max_length))
print(activation_steerer.generate(prompt, max_length=max_length))


def transformation_func(activation_tensor, vec=vector_gpu, c=150):
    return activation_tensor * (1 + (vec.to(activation_tensor.dtype ) * c))

activation_steerer.set_transformation_function(best_layer, transformation_func)

prompt = encourageing_essay_prompts[0]
max_length = 25
print("Good")
print(activation_steerer.generate(prompt, max_length=max_length))
print(activation_steerer.generate(prompt, max_length=max_length))
print(activation_steerer.generate(prompt, max_length=max_length))


def transformation_func(activation_tensor, vec=vector_gpu, c=-150):
    # Add the steering vector (broadcasting should handle dimensions)
    # Ensure dtype compatibility if using AMP

    return activation_tensor * (1 + (vec.to(activation_tensor.dtype ) * c))
activation_steerer.set_transformation_function(best_layer, transformation_func)

prompt = encourageing_essay_prompts[0]
max_length = 25
print("Bad")
print(activation_steerer.generate(prompt, max_length=max_length))
print(activation_steerer.generate(prompt, max_length=max_length))
print(activation_steerer.generate(prompt, max_length=max_length))


and positive.

Wait, the original essay is a bit jumbled. Maybe I should try to fix the structure.

Okay,
and maybe fix the problem.

Wait, the user is talking about the fact that school projects are often teacher-designed, which the
and doesn't make the writer sound like they don't like the reader.

Okay, so I need to write a response to
Good
.

Alright, so if everything is about to be all about the students. If they're going to give students projects over the
.

So, if the student is required to do projects over the summer break. They feel like it's a positive effect on
.

So, wrapping this up, I think the answer is that projects should be student designed because students get to show their full
Bad
.

Wait, but let me think. If a student is given a project that they don't like, they might not like
. But in any case, the project must be student designed, so that the students can do as they please, as long
.

But why would a student not like doing this project? Because the teachers do

In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

from sparsify import SaeConfig, Trainer, TrainConfig
from sparsify.data import chunk_and_tokenize

MODEL = "HuggingFaceTB/SmolLM2-135M"
dataset = load_dataset(
    "EleutherAI/fineweb-edu-dedup-10b", split="train",
)
tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenized = chunk_and_tokenize(dataset, tokenizer)


gpt = AutoModelForCausalLM.from_pretrained(
    MODEL,
    device_map={"": "cuda"},
    torch_dtype=torch.bfloat16,
)

cfg = TrainConfig(SaeConfig(), batch_size=16)
trainer = Trainer(cfg, tokenized, gpt)

trainer.fit()